In [ ]:
import datasets

!export HF_HOME="/home/foresti/.cache/huggingface"
!export HF_DATASET_CACHE="/home/foresti/.cache/huggingface/datasets"
db = datasets.load_dataset("alexfabbri/multi_news", "multi_news")

/home/foresti/miniconda3/envs/mdlm/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for alexfabbri/multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/alexfabbri/multi_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [2]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained(
      "kuleshov-group/caduceus-ph_seqlen-1k_d_model-256_n_layer-4_lr-8e-3", trust_remote_code=True)

vocab = tokenizer.get_vocab()

# Print the first few entries (vocab can be large)
import itertools
print(dict(itertools.islice(vocab.items(), 15)))

{'[CLS]': 0, '[SEP]': 1, '[BOS]': 2, '[MASK]': 3, '[PAD]': 4, '[RESERVED]': 5, '[UNK]': 6, 'A': 7, 'C': 8, 'G': 9, 'T': 10, 'N': 11}


In [2]:
# See dataset structure
print(db)
print(db['train'][0])
print(db['train'][0].keys())
print(db['train'][0]['document'])
print(db['train'][0]['summary'])

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})
{'document': 'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# See the `MDLM` collection page on the hub for list of available models.
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model_name = 'kuleshov-group/mdlm-owt'
model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)

/home/foresti/miniconda3/envs/mdlm/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
import datasets

ds = datasets.load_from_disk("/home/foresti/mdlm/Genomic_Arabidopsis_promoter_dataset_non_tata")

# Print ds information
print(ds)

# Look for None values
# Code to analyze dataset and find null values

# 1. Check column names first
print("Column names:", ds["train"].column_names)

# 2. Check for completely None examples
def check_none_example(example):
    return any(value is None for value in example.values())

# Count examples with at least one None value
none_examples_count = sum(check_none_example(example) for example in ds["train"])
print(f"\nExamples with at least one None value: {none_examples_count}")

# 3. Check for None values in each column
none_values_by_column = {}
for column in ds["train"].column_names:
    none_count = sum(1 for example in ds["train"] if example[column] is None)
    if none_count > 0:
        none_values_by_column[column] = none_count

print("\nNull values by column:")
for column, count in none_values_by_column.items():
    print(f"- {column}: {count} null values ({count/len(ds['train'])*100:.2f}%)")

# 4. Show examples of records with null values (up to 3)
print("\nExamples with null values:")
count = 0
for i, example in enumerate(ds["train"]):
    if check_none_example(example):
        print(f"\nExample {i}:")
        for key, value in example.items():
            print(f"  {key}: {'None' if value is None else (value[:50]+'...' if isinstance(value, str) and len(value) > 50 else value)}")
        count += 1
        if count >= 3:
            break

# 5. Code to filter out rows with None values
print("\nCode to filter out problematic examples:")
print("""
# Filter out examples with None values
def filter_none_values(example):
    return not any(value is None for value in example.values())

# Create filtered dataset
filtered_ds = ds.filter(filter_none_values)

print(f"Original dataset size: {len(ds['train'])}")
print(f"Filtered dataset size: {len(filtered_ds['train'])}")
print(f"Removed {len(ds['train']) - len(filtered_ds['train'])} examples with null values")
""")

DatasetDict({
    train: Dataset({
        features: ['seq', 'description', 'gene_id', 'is_promoter', 'label', 'fp_id', 'range_info'],
        num_rows: 9448
    })
    validation: Dataset({
        features: ['seq', 'description', 'gene_id', 'is_promoter', 'label', 'fp_id', 'range_info'],
        num_rows: 1181
    })
    test: Dataset({
        features: ['seq', 'description', 'gene_id', 'is_promoter', 'label', 'fp_id', 'range_info'],
        num_rows: 1181
    })
})
Column names: ['seq', 'description', 'gene_id', 'is_promoter', 'label', 'fp_id', 'range_info']

Examples with at least one None value: 9448

Null values by column:
- description: 4718 null values (49.94%)
- fp_id: 4730 null values (50.06%)
- range_info: 4730 null values (50.06%)

Examples with null values:

Example 0:
  seq: TTTGATAATATTTTTCTAATGGATACTTGTGTGATTTTGCTTTGCTGGCA...
  description: putative receptor-like protein kinase, F18A8.10
  gene_id: 
  is_promoter: 0
  label: 0
  fp_id: None
  range_info: None

Example 